In [15]:
import re
import math



fp = open("quotes.txt","r")
quotes=[]
globalWordBank=[]


quote=fp.readline()
quote=quote.rstrip('\n')
#Part a) Getting a list of quotes
while quote is not '':
    author=fp.readline()
    author=author.rstrip('\n')
    quotes.append(quote+' - '+author)
    
    
    quote=fp.readline()
    
    quote=quote.rstrip('\n')

    
    



#Part b)    
def quote_splitter(quote):
    quote=quote.lower() 
    #Strips non-alphanumeric characters from the beginning and end of the strings
    while True:
        if re.match('[\w]',quote[0]):
            index=1
            break
        else:
            quote=quote[1:]
    while True:
        if re.match('[\w]',quote[-1]):
            index=1
            break
        else:
            quote=quote[:-1]
    #Splits the quote by any non-alphanumeric characters: We combined contractions and hyphenated
    #words (e.g. isn't = 'isnt')
    quote=re.sub("['`-]",'',quote)
    word_list=re.split('[^\w]+', quote)
    return word_list
    




split_quotes=[]
#Splits all the quotes into lists of words
for quote in quotes:
    
    split_quotes.append(quote_splitter(quote))
    
    #Creates a global word bank: this will be used for the reverse postings-list dictionary
    for word in quote_splitter(quote):
        if word in globalWordBank:
            continue
        else:
            globalWordBank.append(word)

#Part c)
posting_list_dictionary={}
reverse_pld={}
#Goes through each quote
for index in range(len(split_quotes)):
    posting_list_dictionary[quotes[index]]={}
    count=0
    
    #Goes through each word in each quote
    for word in split_quotes[index]:
        count=0
        #We have to add 1 to the length if the index is 0 or length-1 since there will only be a blank space
        #on one side of the word in these cases.
        if split_quotes[index][0]==word:
            count=count+1
        if split_quotes[index][-1]==word:
            count=count+1
        test_quote=quotes[index].lower()
        test_quote=re.sub("['`\"]",'',test_quote)
        instances=re.findall('[^\w]+'+word+'[^\w]+',test_quote)
        #Counts the number of instances that the word shows up in each quote
        count=count+len(instances)
        posting_list_dictionary[quotes[index]][word]=count
        
        
#Part d)
#Creates the reverse postings-list dictionary by checking every quote to see how many times
#every word occurs in it (using the postings-list dictionary to find the number of occurences)
for word in globalWordBank:
    reverse_pld[word]={}
    for index in range(len(split_quotes)):
        if word in split_quotes[index]:
            reverse_pld[word][quotes[index]]=posting_list_dictionary[quotes[index]][word]
        else:
            continue

#Part e)
#TF_IDF function
def TF_IDF(word,quote):
    w=posting_list_dictionary[quote][word]
    #gets a list of every key within the quote
    words_list=[]
    for globalWord in globalWordBank:
        if globalWord in quote_splitter(quote):
            words_list.append(posting_list_dictionary[quote][globalWord])
    #Finds the most often used word within the quote        
    q=max(words_list)
    TF=float(w)/q
    
    total_quotes=len(split_quotes)
    occurences=0
    #Countes the number of occurences of the word in every quote
    for q in quotes:
        if word_in_quote(word,q):
            occurences=occurences+1
            
    IDF=math.log(float(total_quotes)/occurences)
    return TF*IDF

#I made this function to determine if a given word is in a quote, since I test this multiple times
#in the program
def word_in_quote(word,quote):
    if word in quote_splitter(quote):
        return True
    else:
        return False



#Part f)            
#Builds a dictionary for a given word with keys that are quotes that contain the word, and values
#that are the TF_IDF scores of each quote
def TFIDF_dictionary_generator(word):
    dictionary={}
    for quote in quotes:
        if word_in_quote(word,quote):
            dictionary[quote]=TF_IDF(word,quote)
            
    return dictionary

#Part g)
#Builds a dictionary for a given list of words where the keys represent quotes that contain one or more of
#the words and the values represent the sum of TF_IDF scores for each word in the quote
def multiple_word_TFIDF_dictionary_generator(wordList):
    dictionary={}
    for word in wordList:
        for quote in quotes:
            if word_in_quote(word,quote):
                #If the quote is already in the dictionary, then we add the TF_IDF score of the new word
                #(since the quote contains more than one of the given words), otherwise we set it to the
                #TF_IDF score of the first word in that quote
                if quote in dictionary:
                    dictionary[quote]=dictionary[quote]+TF_IDF(word,quote)
                else:
                    dictionary[quote]=TF_IDF(word,quote)
            
    return dictionary




            

            
#print quotes
#print split_quotes
#print globalWordBank
#print posting_list_dictionary
#print reverse_pld

#print TF_IDF('entertainer','An actor is at most a poet and at least an entertainer. - Marlon Brando')
#print TF_IDF('augustine','')
#print TFIDF_dictionary_generator('entertainer')

lisztomania=['heart','mind','disease']
print multiple_word_TFIDF_dictionary_generator(lisztomania)
print TF_IDF('heart','The heart has its reasons, of which the mind knows nothing. - Blaise Pascal')+TF_IDF('mind','The heart has its reasons, of which the mind knows nothing. - Blaise Pascal')


{'As every divided kingdom falls, so every mind divided between many studies confounds and saps itself. - Leonardo da Vinci': 2.0393333747514353, 'No passion so effectually robs the mind of all its powers of acting and reasoning as fear. - Edmund Burke': 2.0393333747514353, 'In every man`s heart there is a secret nerve that answers to the vibrations of beauty. - Christopher Morley': 4.484131857611035, 'The mind of the thoroughly well-informed man is a dreadful thing. It is like a bric-\xe0-brac shop, all monsters and dust, with everything priced above its proper value. - Oscar Wilde': 2.0393333747514353, 'What we are today comes from our thoughts of yesterday and our present thoughts build our life tomorrow. Our life is the creation of our mind. - Siddhartha Buddha': 0.8157333499005741, 'Too many poets delude themselves by thinking that the mind is dangerous and must be left out. Well, the mind is dangerous and must be left in. - Robert Frost': 4.078666749502871, 'Love anything and you